In [121]:
'''
Test hypothesis that if VIX (cost of S&P 500 options), VVIX (cost of VIX options) and S&P 500 all decline,the market moves
higher in the following days. 

'''

# Import libraries

import yfinance as yf # YFinance wrapper to retrieve prices
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
%matplotlib inline

In [122]:
# import data
tickers=['^VIX','^VVIX','^GSPC']
start='2007-01-01' # VVIX prices start in 2007

data=pd.DataFrame()
for t in tickers:
    data[t]=yf.download(t,start=start)['Adj Close'] #onlt need Adjusted Close

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [123]:
# Inspect data
data.head()

,^VIX,^VVIX,^GSPC
Date,,,
2007-01-03,12.04,87.629997,1416.599976
2007-01-04,11.51,88.190002,1418.339966
2007-01-05,12.14,90.169998,1409.709961
2007-01-08,12.00,92.040001,1412.839966
2007-01-09,11.91,92.760002,1412.109985


In [124]:
# Inspect data
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3416 entries, 2007-01-03 to 2020-07-28
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ^VIX    3416 non-null   float64
 1   ^VVIX   3413 non-null   float64
 2   ^GSPC   3416 non-null   float64
dtypes: float64(3)
memory usage: 106.8 KB


In [125]:
# Check null values and adjust
check_nan=data.isnull()
check_nan.loc[check_nan['^VVIX']==True]

,^VIX,^VVIX,^GSPC
Date,,,
2010-11-11,False,True,False
2019-07-05,False,True,False
2020-06-11,False,True,False


In [126]:
#fill null value with forward fill method and rename columns
data['^VVIX'].fillna(method='ffill',inplace=True)
data.rename(columns={'^VIX':'vix','^VVIX':'VVIX','^GSPC':'spx'},inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3416 entries, 2007-01-03 to 2020-07-28
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   vix     3416 non-null   float64
 1   VVIX    3416 non-null   float64
 2   spx     3416 non-null   float64
dtypes: float64(3)
memory usage: 106.8 KB


In [127]:
#Calculate returns for VIX, VVIX, SPX
for col in data.columns:
    data['{}_ret'.format(col)]=data[col].pct_change()
data.head()

,vix,VVIX,spx,vix_ret,VVIX_ret,spx_ret
Date,,,,,,
2007-01-03,12.04,87.629997,1416.599976,NaN,NaN,NaN
2007-01-04,11.51,88.190002,1418.339966,-0.044020,0.006391,0.001228
2007-01-05,12.14,90.169998,1409.709961,0.054735,0.022451,-0.006085
2007-01-08,12.00,92.040001,1412.839966,-0.011532,0.020739,0.002220
2007-01-09,11.91,92.760002,1412.109985,-0.007500,0.007823,-0.000517


In [128]:
# Calculate future returns for spx (split into 2 steps in order to view future prices as well)
# 1. Add spx future price columns using .shift(-period) method

periods=[1,3,5] # Define forward days (periods)

for days in periods:
    data['fut_price_{}'.format(days)]=data['spx'].shift(-days)
data.tail()

,vix,VVIX,spx,vix_ret,VVIX_ret,spx_ret,fut_price_1,fut_price_3,fut_price_5
Date,,,,,,,,,
2020-07-22,24.32,112.839996,3276.020020,-0.020934,-0.022692,0.005747,3235.659912,3239.409912,NaN
2020-07-23,26.08,116.699997,3235.659912,0.072368,0.034208,-0.012320,3215.629883,3239.399902,NaN
2020-07-24,25.84,117.220001,3215.629883,-0.009202,0.004456,-0.006190,3239.409912,NaN,NaN
2020-07-27,24.74,112.709999,3239.409912,-0.042570,-0.038475,0.007395,3239.399902,NaN,NaN
2020-07-28,24.18,110.559998,3239.399902,-0.022635,-0.019076,-0.000003,NaN,NaN,NaN


In [129]:
# 2. calculate future returns for spx using .pct_change(period) method
for days in periods:
    data['spx_fut_rtn_{}'.format(days)]=data['fut_price_{}'.format(days)].pct_change(days)
data.tail()

,vix,VVIX,spx,vix_ret,VVIX_ret,spx_ret,fut_price_1,fut_price_3,fut_price_5,spx_fut_rtn_1,spx_fut_rtn_3,spx_fut_rtn_5
Date,,,,,,,,,,,,
2020-07-22,24.32,112.839996,3276.020020,-0.020934,-0.022692,0.005747,3235.659912,3239.409912,NaN,-0.012320,-0.011175,-0.011178
2020-07-23,26.08,116.699997,3235.659912,0.072368,0.034208,-0.012320,3215.629883,3239.399902,NaN,-0.006190,0.001156,0.001156
2020-07-24,25.84,117.220001,3215.629883,-0.009202,0.004456,-0.006190,3239.409912,NaN,NaN,0.007395,0.007392,0.007392
2020-07-27,24.74,112.709999,3239.409912,-0.042570,-0.038475,0.007395,3239.399902,NaN,NaN,-0.000003,-0.000003,-0.000003
2020-07-28,24.18,110.559998,3239.399902,-0.022635,-0.019076,-0.000003,NaN,NaN,NaN,0.000000,0.000000,0.000000


In [130]:
# Drop null value and inspect data
data.dropna(inplace=True) 
summary=data.describe()
summary

,vix,VVIX,spx,vix_ret,VVIX_ret,spx_ret,fut_price_1,fut_price_3,fut_price_5,spx_fut_rtn_1,spx_fut_rtn_3,spx_fut_rtn_5
count,3406.000000,3406.000000,3406.000000,3406.000000,3406.000000,3406.000000,3406.000000,3406.000000,3406.000000,3406.000000,3406.000000,3406.000000
mean,19.806941,90.541254,1840.273822,0.003353,0.001372,0.000333,1840.820260,1841.876261,1842.938002,0.000334,0.000937,0.001542
std,9.844190,15.353380,653.538567,0.082523,0.052367,0.013184,653.960703,654.743852,655.542933,0.013184,0.020470,0.025752
min,9.140000,59.740002,676.530029,-0.295727,-0.210547,-0.119841,676.530029,676.530029,676.530029,-0.119841,-0.139059,-0.183401
25%,13.480000,80.752501,1315.152496,-0.042806,-0.028096,-0.003982,1315.152496,1315.152496,1315.152496,-0.003982,-0.007479,-0.009007
50%,16.790001,88.349998,1721.940002,-0.006100,-0.004061,0.000687,1723.929993,1737.520020,1744.580017,0.000687,0.002422,0.003663
75%,22.719999,97.709999,2348.142456,0.036299,0.024070,0.005722,2348.629944,2349.190002,2351.144958,0.005726,0.010850,0.014532
max,82.690002,207.589996,3386.149902,1.155979,0.570027,0.115800,3386.149902,3386.149902,3386.149902,0.115800,0.175503,0.191112


In [131]:
### Hypothesis Testing ####
mask_1=data['vix_ret']<0 # VIX return negative
mask_2=data['VVIX_ret']<0 # VVIX return negative
mask_3=data['spx_ret']<0 #SPX return negative

filtered_df=data[['spx_fut_rtn_1','spx_fut_rtn_3','spx_fut_rtn_5']].loc[mask_1&mask_2&mask_3]
filtered_summary=filtered_df.describe()
filtered_summary # Summary dataframe of SPX future returns (periods 1,2,3) given conditions (masks) 1,2,3

,spx_fut_rtn_1,spx_fut_rtn_3,spx_fut_rtn_5
count,209.000000,209.000000,209.000000
mean,0.000620,0.001765,0.002603
std,0.014716,0.025914,0.030917
min,-0.044163,-0.103973,-0.181955
25%,-0.005676,-0.010864,-0.010234
50%,0.001186,0.001966,0.004295
75%,0.006127,0.011518,0.016747
max,0.093828,0.175503,0.173974


In [132]:
for period in periods:
    wins=filtered_df['spx_fut_rtn_{}'.format(period)].loc[filtered_df['spx_fut_rtn_{}'.format(period)]>0].count()
    percent_winners=round(wins/filtered_df['spx_fut_rtn_{}'.format(period)].count(),2)*100
    max_gain=filtered_df['spx_fut_rtn_{}'.format(period)].loc[filtered_df['spx_fut_rtn_{}'.format(period)]>0].max()*100
    max_loss=filtered_df['spx_fut_rtn_{}'.format(period)].loc[filtered_df['spx_fut_rtn_{}'.format(period)]<0].min()*100
    avg_gain=filtered_df['spx_fut_rtn_{}'.format(period)].loc[filtered_df['spx_fut_rtn_{}'.format(period)]>0].mean()*100
    avg_loss=filtered_df['spx_fut_rtn_{}'.format(period)].loc[filtered_df['spx_fut_rtn_{}'.format(period)]<0].mean()*100
    
    print('{}-DAYS FORWARD: '.format(period))
    print('Pct winners: '+str(percent_winners)+'%')
    print('Max Gain: '+str(round(max_gain,2))+'%')
    print('Max Loss: '+str(round(max_loss,2))+'%')
    print('Avg Gain: '+str(round(avg_gain,2))+'%')
    print('Avg Loss: '+str(round(avg_loss,2))+'%')
    print('')

1-DAYS FORWARD: 
Pct winners: 56.99999999999999%
Max Gain: 9.38%
Max Loss: -4.42%
Avg Gain: 0.87%
Avg Loss: -1.01%

3-DAYS FORWARD: 
Pct winners: 57.99999999999999%
Max Gain: 17.55%
Max Loss: -10.4%
Avg Gain: 1.58%
Avg Loss: -1.79%

5-DAYS FORWARD: 
Pct winners: 59.0%
Max Gain: 17.4%
Max Loss: -18.2%
Avg Gain: 1.9%
Avg Loss: -2.13%

